In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
from bs4 import BeautifulSoup
import time
import pickle
from selenium.webdriver.common.by import By


In [2]:
# Set path Selenium
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
s = Service(CHROMEDRIVER_PATH)
WINDOW_SIZE = "1920,1080"

In [3]:
# Options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=s, options=chrome_options)


# Funções

In [22]:
# Função para carregar os cookies
def load_cookies(path):
    # Carregar os cookies
    cookies = pickle.load(open(path+"cookies_estadao.pkl", "rb"))
    # Adicionar os cookies ao navegador
    for cookie in cookies:
        driver.add_cookie(cookie)
    return

# Função para carregar mais publicações do blog
def carregar_mais(xpath, v):
    # Clicar no botão v vezes usando uma variável de controle
    controle = 0
    while controle < v:
        # Clicar no botão "Carregar mais"
        botao = driver.find_element(By.XPATH, xpath)
        botao.click()
        # Esperar a página carregar
        time.sleep(5)
        # Atualizar o controle
        controle += 1
    return

In [5]:
def get_urls(url, class_, path, botao_carregar=None, qntd_carregar=None):
    print(f'Recolhendo links ...')

    driver.get(url)

    if botao_carregar != None:
        load_cookies(path)
        carregar_mais(botao_carregar, qntd_carregar)

    # Encontrar todos os links dentro da div com a classe "thumbnails-wrapper"
    elementos_links = driver.find_elements("xpath", f"//div[@class='{class_}']//a")

    # Extrair os links (atributo href) de cada elemento encontrado
    links = [elemento.get_attribute('href') for elemento in elementos_links]

    # retirando links repetidos
    links = list(set(links))

    return links

In [18]:
def get_content(links, class_text):
    conteudo = dict()
    
    for url in links:
        print(f'Recolhendo dados {url}...')
        # Get the response and print title
        driver.get(url)
        time.sleep(2)

        # Extrair o conteúdo da página
        conteudo_html = driver.page_source

        # Utilizar BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(conteudo_html, 'html.parser')

        # Encontrar a div com a classe "entry-content"
        div_entry_content = soup.find('div', class_=class_text)

        if div_entry_content != None:
            # Encontrar a tag <body> e extrair apenas o texto
            conteudo[url] = div_entry_content.get_text(strip=True)
        else:
            print('vazio')

    return conteudo

In [19]:
path = '../data/'

# uol.com.br/nossa/

In [30]:
url_principal = "https://www.uol.com.br/nossa/cozinha/tim-tim/"
class_ = 'flex-wrap '

links = get_urls(url_principal, class_, path)


class_text = 'text'
conteudo = get_content(links, class_text)

Recolhendo links ...
Recolhendo dados https://www.uol.com.br/nossa/noticias/redacao/2023/06/07/vinho-brasileiro-conquista-medalha-em-concurso-internacional-pela-1-vez.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/redacao/2023/06/01/franca-destroi-35-mil-garrafas-de-soda-pela-audacia-de-se-dizer-champagne.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/afp/2023/06/01/clara-de-ovo-no-vinho-produtores-terao-que-explicar-ingredientes-no-rotulo.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/ansa/2023/05/30/spritz-e-drink-favorito-dos-italianos-no-aperitivo.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/agencia-estado/2023/05/24/prefeitura-sanciona-lei-para-combater-assedio-sexual-em-bares-e-restaurantes-de-sp.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/redacao/2023/05/26/vai-um-gim-tonica-raiz-como-drinques-famosos-mudaram-com-o-tempo.htm...
Recolhendo dados https://www.uol.com.br/nossa/noticias/redacao/2023/06/02

In [31]:
df = pd.DataFrame({'URL': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_uol_timtim.csv')

# Suzana Barelli

In [20]:
url_principal = "https://neofeed.com.br/autor/suzana-barelli/"
class_ = 'td_module_10 td_module_wrap td-animation-stack '

links = get_urls(url_principal, class_, path)

# retirando links indesejados
links = list(set(links))

for l in links:
    if len(l) <= 50:
        links.remove(l)


class_text = 'td-post-content tagdiv-type'
conteudo = get_content(links, class_text)

In [16]:
df = pd.DataFrame({'URL': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_Suzana_Barelli.csv')

# Balcão do Giba (Coluna Estadão)

In [23]:
url_principal = "https://www.estadao.com.br/paladar/balcao-do-giba/"
class_ = 'content'
botao_carregar = '//*[@id="fusion-app"]/div/div[1]/div/div/div/div[1]/div/div[2]/div/div[3]/button'
qntd_carregar = 5

links = get_urls(url_principal, class_, path, botao_carregar, qntd_carregar)

class_text = 'styles__ContentWrapperContainerStyled-sc-1ehbu6v-0 klsZKo content-wrapper news-body container content template-reportagem already-sliced already-checked'
conteudo = get_content(links, class_text)

Recolhendo links ...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/loop-abre-as-portas-na-vila-madalena-com-foco-na-sustentabilidade/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/com-fiica-bartender-toalha-realiza-o-sonho-da-casa-propria/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/nao-sabe-o-que-dar-de-presente-no-dia-das-maes-que-tal-preparar-um-coquetel-para-a-mulher-mais-importante-da-sua-vida/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/drinque-em-homenagem-ao-gabriel-garcia-marquez-no-bar-riviera/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/conheca-os-bares-de-coquetelaria-que-promovem-bailinhos-de-carnaval/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/greenkitchen-volta-com-coqueteis-de-thiago-pereira-conheca-tambem-os-drinques-da-adega-santiago/...
Recolhendo dados https://www.estadao.com.br/paladar/balcao-do-giba/no-gran-bar-exper

In [24]:
df = pd.DataFrame({'URL': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_Balcão_do_Giba.csv')

# Só queijo (Coluna Estadão)

In [25]:
url_principal = "https://www.estadao.com.br/paladar/so-queijo/"
class_ = 'content'

links = get_urls(url_principal, class_, path, botao_carregar, qntd_carregar)

class_text = 'styles__ContentWrapperContainerStyled-sc-1ehbu6v-0 klsZKo content-wrapper news-body container content template-reportagem already-sliced already-checked'
conteudo = get_content(links, class_text)

Recolhendo links ...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/queijistas-em-cena-no-palco-da-faria-lima/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/encantou-se-gabriel-andrade-defensor-do-queijo-de-leite-cru/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/revisitando-o-queijo-minas-artesanal/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/aventura-queijeira-na-polonia/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/concurso-de-melhor-fondue-do-brasil-em-sao-paulo/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/repercussoes-do-mundial-do-queijo-do-brasil-em-sao-paulo/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/no-tabuleiro-da-baiana-tem-queijo/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/a-ressurreicao-de-266-kg-queijos-artesanais-paulistas-enterrados-vivos/...
Recolhendo dados https://www.estadao.com.br/paladar/so-queijo/sial-em-p

In [26]:
df = pd.DataFrame({'URL': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_So_queijo.csv')

# Blog da Belle (Coluna Estadão)

In [27]:
url_principal = "https://www.estadao.com.br/paladar/blog-da-belle/"
class_ = 'content'

links = get_urls(url_principal, class_, path, botao_carregar, qntd_carregar)

class_text = 'styles__ContentWrapperContainerStyled-sc-1ehbu6v-0 klsZKo content-wrapper news-body container content template-reportagem already-sliced already-checked'
conteudo = get_content(links, class_text)

Recolhendo links ...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/pirulito-de-vinho-existe-coisa-de-americano-mas-entrega-no-brasil/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/romanee-conti-com-oferta-inicial-de-r-21-mil-e-destaque-de-leilao/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/produtores-de-vinho-e-de-cannabis-testam-colaboracao-na-california/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/incendio-atinge-100-mil-hectares-na-california-deixa-mortos-e-destroi-vinicolas/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/bares-criam-festival-do-vinho-e-vendem-rotulos-a-preco-de-importadora-durante-dez-dias/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/pequeno-glossario-do-vinho-2-vinho-vinicola-vinhedo-e-outras-palavras-com-v/...
Recolhendo dados https://www.estadao.com.br/paladar/blog-da-belle/vinho-natural-apreendido-no-rs-sera-analisado/...
R

In [28]:
df = pd.DataFrame({'URL': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_Blog_da_Belle.csv')

# vinhonosso (blog)

In [29]:
datas = [
    ['2018', '12'] ,['2021', '09']  , ['2018', '02'], ['2018', '01'],
    ['2017', '10'], ['2017', '08'], ['2017', '07'], ['2017', '05']
    ]

conteudo = dict()

for ano, mes in datas:
    print(f'Recolhendo dados {ano}/{mes}...')
    # Get the response and print title
    driver.get(f"https://vinhonosso.com/{ano}/{mes}/")

    # Extrair o conteúdo da página
    conteudo_html = driver.page_source

    # Utilizar BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(conteudo_html, 'html.parser')

    # Encontrar a div com a classe "entry-content"
    div_entry_content = soup.find('div', class_='entry-content')

    # Encontrar a tag <body> e extrair apenas o texto
    conteudo[f'{ano}/{mes}'] = div_entry_content.get_text(strip=True)

df = pd.DataFrame({'Data': conteudo.keys(), 'Conteúdo': conteudo.values()})
df.to_csv(path+'conteudo_vinhonosso.csv')

Recolhendo dados 2018/12...
Recolhendo dados 2021/09...
Recolhendo dados 2018/02...
Recolhendo dados 2018/01...
Recolhendo dados 2017/10...
Recolhendo dados 2017/08...
Recolhendo dados 2017/07...
Recolhendo dados 2017/05...
